In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp cli

In [ ]:
from fastcore.script import *

from rough.data import *
from rough.profile import *
from rough.section import *

In [ ]:
@call_parse
def main(msg:str,     # The message
         upper:bool): # Convert to uppercase?
    "Print `msg`, optionally converting to uppercase"
    print(msg.upper() if upper else msg)

In [ ]:
main('Bingo',upper=False)

Bingo


Hello world


In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()